# Milestone 1: CS207 Group 21


# Introduction

Describe the problem the software solves and why it's important to solve that problem.

Differentiation has ubiquitous applications in many areas of mathematics, sciences and engineering. As such, it is certainly useful and convenient if computer programs could carry out differentiation automatically for application in a wide variety of cases. For computationally heavy projects, the ability to compute derivatives automatically becomes even more critical as manually working out deriatives in such projects is certainly an impossible task. Even though there exists methods such as *numerical diffentiation* and *symbolic differentiation* in determining derivatives computationally, these two methods have their limitations. In the following, we shall briefly review *numerical diffentiation* and *symbolic differentiation* to highlight some of their difficulties before moving on to describing *automatic differentiation* and the advantages it brings over the other two methods.   

### Numerical Differentiation
In *numerical differentiation*, the value of derivatives is approximated using the following formula:

$$
\frac{\partial{f(x)}}{\partial{x}} \approx \frac{f(x+h)-f(x)}{h}
$$

However, when the h values are too small, the numerical approximation fluctuates about the analytical answer. This is because the step size is too small, leading to a round-off error of the floating points caused by the limited precision of computations. On the other hand, when the h values are too large, the numerical approximation becomes inaccurate. This is because the step size is too big, leading to an error of approximation known as truncation error.

### Symbolic Differentiation
In *symbolic differentiation*, expressions are manipulated automatically to obtain the required derivatives. At its heart, *symbolic differentiation* applies transformations that captures the various rules of differentiation in order to manipulate the expressions. However, *symbolic differentiation* requires careful and sometimes ingenious design as accidental manipulation can easily produce large expressions which take up a lot of computational power and time, which leads to a problem known as expression swell.

### Automatic Differentiation
As seen from above, both *numerical diffentiation* and *symbolic differentiation* have their respective issues when it comes to computing derivatives. These issues are further exacerbated when calculating higher order derivatives, where both errors and complexity increases. *Automatic differentiation* overcomes these issues by recognizing that every differentiation, no matter how complicated, can be executed in a stepwise fashion with each step being an execution of either the elementary arithmetic operations (addition, substraction, multiplication, division) or the elementary functions (sin, sqrt, exp, log, etc.). To track the evaluation of each step, *automatic differentiation* produces computational graphs and evaluation traces. To compute the derivatives, *automatic differentiation* applies the chain rule repeatedly at all steps. By taking a stepwise approach and using the chain rule, *automatic differentiation* circumvents the issues encountered by both *numerical diffentiation* and *symbolic differentiation* and automatically compute derivatives that are both accurate and with a high level of precision. In order to further understand *automatic differentiation*, we present the mathematical background and essential ideas of *automatic differentiation* in the next section.

Note - In our research of automatic differentiation, we referred to the following resources:

Baydin, A.G., Pearlmutter, B.A., Radul, A. A. & Siskind, J.M. (2018). Automatic differentiation in machine learning: A survey. *Journal of Machine Learning Research, 18*, 1-42.

Geeraert, S., Lehalle, C.A., Pearlmutter, B., Pironneau, O. & Reghai, A. (2017). Mini-symposium on automatic differentiation and its applications in the financial industry. *ESAIM: Proceedings and Surverys* (pp. 1-10).

Berland, H. (2006). *Automatic differentiation* [PowerPoint Slides]. Retrieved from http://www.robots.ox.ac.uk/~tvg/publications/talks/autodiff.pdf

# Background


As mentioned before, *automatic differentiation* employs a stepwise approach and chain rule to automatically compute derivatives. We shall first state the chain rule in calculus before showing an example production of an evaluation trace and computational graph. Next, we discuss one mode of *automatic differentiation*, namely the forward mode. In particular, the demonstration of the use of chain rule at each step to determine derivatives will be shown here. Finally, we touch on the use of dual numbers in *automatic differentiation*. 

### Chain Rule 
For a function $f(u(t),v(t))$, the chain rule is given by

$$
\begin{align}
 \frac{\partial f}{\partial t} = \frac{\partial f}{\partial u}\frac{\partial u}{\partial t} + \frac{\partial f}{\partial v}\frac{\partial v}{\partial t}
\end{align}
$$

### Example Production of Evaluation Trace & Computational Graph
The most straightforward way to show the generation of an evaluation trace and computational graph is to consider an example. For this purpose, we study the following function 

$$
f(x,y) = sin(x) + 4y
$$

#### Evaluation Trace
The evaluation trace breaks the function into individual steps and creates a buildup of the function starting with the input variables. At each step, only either an elementary arithmetic operation (addition, substraction, multiplication, division) or an elementary function (sin, sqrt, exp, log, etc.) is used to build the function for the next step. The evaluation trace for our function of interest is shown in the table below.

| Trace | Elementary Function | Current Value | Comment               | 
| :---: | :-----------------: | :-----------: | :-------------------: | 
| $x_{1}$ | $x_{1}$           | $x$           | Input x               |
| $x_{2}$ | $x_{2}$           | $y$           | Input y               |
| $x_{3}$ | $sin(x_{1})$      | $sin(x)$      | Elementary function   |
| $x_{4}$ | $4*x_{2}$         | $4y$          | Elementary arithmetic |
| $x_{5}$ | $x_{3}+x_{4}$     | $sin(x) + 4y$ | Elementary arithmetic |


#### Computational Graph 
The computational graph translates the essence of the evaluation trace into a graph and captures the relationship between each step. Refer to the figure below for the computational graph of our function of interest.  

![computational-graph](Computational_Graph.png)

### Forward Mode
Armed with the knowledge of the chain rule, evaluation trace and computational graph, we can now consider the forward mode of *automatic differentiation*. The table below shows the earlier evaluation trace table that has now been expanded to include columns that store derivatives. At each step, the chain rule is applied to the elementary function to determine the elementary function derivative.

For instance, 

Trace $x_{3}$

$$
\begin{align}
\dot{x_{3}} &= \frac{\partial{sin(x_{1})}}{\partial{x_{1}}} \dot{x}_{1} \\
&= cos(x_{1})\dot{x}_{1}
\end{align} 
$$

Trace $x_{5}$
$$
\begin{align}
\dot{x_{5}} &= \frac{\partial{(x_{3}+x_{4}})}{\partial{x_{3}}} \dot{x}_{3} +  \frac{\partial{(x_{3}+x_{4}})}{\partial{x_{3}}} \dot{x}_{4} \\
&= \dot{x}_{3}+\dot{x}_{4}
\end{align} 
$$

| Trace | Elementary Function | Current Value | Elementary Function Derivative | $\nabla_{x}$ Value  | $\nabla_{y}$ Value  | 
| :---: | :-----------------: | :-----------: | :--------------------------: | :---------------------: | :---------------------: | 
| $x_{1}$ | $x_{1}$       | $x$           | $\dot{x}_{1}$             | $1$      | $0$ |
| $x_{2}$ | $x_{2}$       | $y$           | $\dot{x}_{2}$             | $0$      | $1$ |
| $x_{3}$ | $sin(x_{1})$  | $sin(x)$      | $cos(x_{1})\dot{x}_{1}$   | $cos(x)$ | $0$ |
| $x_{4}$ | $4*x_{2}$     | $4y$          | $4\dot{x}_{2}$            | $0$      | $4$ |
| $x_{5}$ | $x_{3}+x_{4}$ | $sin(x) + 4y$ | $\dot{x}_{3}+\dot{x}_{4}$ | $cos(x)$ | $4$ |

As seen from the table above, the derivative of elementary functions such as $sin$ has to be done manually and this has implications for our design of the *automatic differentiation* package later. Specifically speaking, we would need to define separate classes for each elementary function. For more details, refer to the Implementation section below.

In addition, the first and second row has initial values for $\nabla_{x}$ and $\nabla_{y}$ as (1,0) and (0,1) respectively. These are actually seed values for the stepwise propagation of the values of derivatives. The forward mode actually calculates the dot product between the gradient of our function with the seed vector (ie directional derivative). In this case, we have a scalar function with two variables, but in the case of a vector function of vectors, the forward mode actually calculates the dot product between the Jacobian matrix ($J$) and seed vector ($p$) (ie $J.p$). 

### Dual Numbers
Dual numbers extend the real number line in another direction by adding a second component. This extension is analagous to the extension of real numbers by imaginary numbers. The general form of a dual number is given by 

$$ x = a + \epsilon b, $$

where $\epsilon$ is defined as $\epsilon^2 = 0$, $a$ is the real part and $b$ is the dual part of the dual number.

In our *automatic differentiation* package, we can define a dual class that has two attributes. One of these attributes stores the value of the function while the other stores the value of the derivatives. This is similar to having a dual number with the value of a function as the real part and the value of derivatives as the dual part. Having such a dual number structure allows us to carry out the expected arithmetic operations between two dual instances.

#### Addition

$$ 
\begin{align}
(x +\epsilon \dot{x}) + (y +\epsilon \dot{y}) &= (x+y) + \epsilon(\dot{x}+\dot{y})
\end{align}
$$ 

#### Subtraction

$$ 
\begin{align}
(x +\epsilon \dot{x}) - (y +\epsilon \dot{y}) &= (x-y) + \epsilon(\dot{x}-\dot{y})
\end{align}
$$ 

#### Multiplication

$$ 
\begin{align}
(x +\epsilon \dot{x})*(y +\epsilon \dot{y}) &= xy+\epsilon x\dot{y}+\epsilon \dot{x}y+\epsilon^2\dot{x}\dot{y}\\
&= xy + \epsilon(x\dot{y} + \dot{x}y)
\end{align}
$$ 

#### Division

$$ 
\begin{align}
(x +\epsilon \dot{x}) / (y +\epsilon \dot{y}) &= \frac{(x +\epsilon \dot{x})(y -\epsilon \dot{y})}{(y +\epsilon \dot{y})(y - \epsilon \dot{y})} \\
&= \frac{xy-\epsilon x\dot{y}+\epsilon \dot{x}y-\epsilon^2\dot{x}\dot{y}}{y^2-\epsilon^2\dot{y}^2} \\
&= \frac{xy + \epsilon(-x\dot{y} + \dot{x}y)}{y^2} \\
&= \frac{x}{y} + \frac{\epsilon(y\dot{x}-x\dot{y})}{y^2} 
\end{align}
$$ 

In sum, this section covers the mathematical background and essential ideas of *automatic differentiation* for a scalar function with two variables. These basic concepts can be extended easily to higher dimensions if needed. In fact, our *automatic differentiation* package will not only handle scalar functions of scalar and vector values, but also vector functions of vectors.

# How to Use Package Name
The purpose of this section is to give a sketch of how a user can interact with the package.

We envision that users who want to use our package can use “pip install” to install the package, and we will provide the “requirements.txt” to help users create the correct environment.  We will use requirements.txt instead of a virtual environment to make our package more accessible to users who may not be familiar with virtual environments.  If one wishes to set up a virtual environment, he or she still can and interact with the package as needed.

We will create a standard structure of package files as below:
```
PackageAD/
  AD/
      ...
  README.md
  setup.py
  LICENSE
  requirements.txt
```
AD/ is the project directory where our package will locate. In README.md, we will describe the install process and usage of our package. Also we will give some examples about how to use the package in README.md. setup.py is the build script for our package and LICENSE is the document that tells the users how to use and under what conditions they can use our package.

The package will have one module named AD. Users can interact with our package as follows:


## Example 1: Scalar-Valued Function of Scalar
The user should _always_ set up variables along with their values using the DualNumber class (for now).  The user can then perform operations and elementary functions on this variable.

```python
from AD.DualNumber import DualNumber
from AD.ElementaryFunctions import ElemFunctions
# DualNumber is a class in module AD, user must. The value can be a vector or a number.
x = DualNumber(5, 'x')

# ElemFunctions is a class where we define some elementary function derivatives and calculate the derivative function.
func = ElemFunctions.sin(x)

# we can get the value and derivative from the attributes "value" and "der". If we did not assign value and derivative direction in the fist
# step, we can do it here. 
print(func.value)
print(func.der)
    
```


## Example 2: Scalar-Valued Function of Vector
The user should _always_ set up variables along with their values using the DualNumber class.  With more than one variable, the user should first define each variable and then user the get_der method to get a specific partial derivative.  The user can also specify a direction to get a directional derivative.

```python
from AD.DualNumber import DualNumber
from AD.ElementaryFunctions import ElemFunctions
# DualNumber is a class in module AD, user must. The value can be a vector or a number.
x = DualNumber(5, 'x')
y = DualNumber(6, 'y')

# ElemFunctions is a class where we define some elementary function derivatives and calculate the derivative function.
func = ElemFunctions.sin(x*y)

# we can get the value and derivative from the attributes "value" and "der". If we did not assign value and derivative direction in the fist
# step, we can do it here. 
print(func.value)

# get partial with respect to x
print(func.get_der('x'))
# get directional derivative
print(func.get_der(direction = {'x': 1, 'y': 6}))
    
```

## Example 3: Vector-Valued Function of Vector
For vector-valued functions, we can use the class Parallelized as follows:
```python
from AD.DualNumber import DualNumber
from AD.ElementaryFunctions import ElemFunctions
# DualNumber is a class in module AD, user must. The value can be a vector or a number.
x = DualNumber(5, 'x')
y = DualNumber(6, 'y')

# ElemFunctions is a class where we define some elementary function derivatives and calculate the derivative function.
# Parallelized is a class which takes a vector-valued output
func = Parallelized([ElemFunctions.sin(x*y), ElemFunctions.cos(x*y)])

# we can get the value and derivative from the attributes "value" and "der". If we did not assign value and derivative direction in the fist step, do it here

print(func.value([1,0]))

# get gradient with respect to specific input
print(func.get_der('x', 1))

# dot the Jacobian with a vector p to get a directional gradient
print(func.get_der(direction = {'x': 1, 'y': 6}))
    
```

In general, users should always:

1. Initialize all variables (e.g. x, y) using a value and a name for each.  Note that the requirement that the user input a value during initialization is a requirement for _now_, but we will add functionality for inputting a value after declaring the variable and function in our final product.
2. Use elementary functions and operators as needed.
3. Use get_grad for vector-valued inputs (with an argument for direction and/or partial) and .der for scalar-valued inputs.

# Software Organization

## Directory Structure
Our directory structure will be as follows (this is done with help from [Packaging](https://python-packaging.readthedocs.io/en/latest/index.html)):
```
AutoDiff/
│   README.md
│   LICENSE    
│   setup.py 
│   requirements.txt
│
└───AD/
│   │   __init__.py
│   │   DualNumber.py
│   │   ElementaryFunctions.py (each function implemented as a method of a class)
│   └───tests/
│       │   ...
│   └───visualizations/
│       │   PostProcess.py
│
│    examples.py
```

## Modules

### DualNumber
DualNumber will allow the user to construct variables by inputting a value and a variable name.  It will overload operators to allow efficient adding, subtracting, etc of variables.

### ElementaryFunctions
ElementaryFunctions will allow the user to compute elementary functions of variables defined with DualNumber.  For example, a user may specify x=DualNumber('x', 5), function = ElementaryFunctions.ElemFunction.sin(x), function.der to compute the derivative of sin(x) at 5.

### tests
The tests module will contain test suites for each of the classes and will use pytest for this testing.

### visualizations
The visualizations module will have capability to generate computational graphs and the evaluation trace.

## Test Suite
We will use TravisCI and CodeCov for continuous integration, and our test suite will be in the /tests subdirectory.

## Distribution
We will distribute our package using PyPI to allow our users to use pip to install our package.  We will not use any specific frameworks as through online reading there seem to be no particular benefits to using frameworks (frameworks are often used for web hosting and creating web applications, neither of which we currently plan to do).

# Implementation

## Classes and Data Structures

After reviewing the literature ([1](https://arxiv.org/pdf/1811.05031.pdf), [2](http://www.jmlr.org/papers/volume18/17-468/17-468.pdf), and [3](https://www.mcs.anl.gov/papers/P1152.pdf)), we concluded that we'd make use of _operator overloading_ for implementing dual numbers for the forward mode of automatic differentiation.


### DualNumber()
The DualNumber() superclass will make up the core of our setup. The user may interact with the class to set up scalar- or vector-valued variables that will later be inputed into functions.  When constructing a DualNumber object, the user should _always_ specify a value and variable name (as a string) for the object as follows:

1. x = DualNumber(5, 'x')

DualNumber will then store the variable name, 'x', and its value, 5 in the DualNumber data structure.  DualNumber will also be used to overload basic operations such as adding, subtracting, multiplying, and dividing.  

To demonstrate the functionality of this class, let's focus on the overloaded \_\_add__ operator. Note that there are two regimes we should consider: (1) the case in which both the operands are the same variable (scalar-valued), (2) the case in which the operands are different variables.  The first case is relatively straightforward and we simply handle it by adding the derivatives and the values.  So, if the function is $f(x) = x^2 + \sin(x)$ evaluated at $x=5$, we just add the two functions' values $5^2 + \sin(5)$ and their derivatives. Here, it would look as follows:

The principles guiding this software design are grounded in its simplicity and robustness.  

1. Each object in our automatic differentiation scheme will inherit from DualNumber() (for example, ElemFunctions(), explained below, is just an extension of DualNumber()) since all objects will have a value and gradient. 
2. Using the dictionary datatype to keep track of partial derivatives will allow the user clarity in specifying which variables to differentiate.  Theoretically, we could base this entire process in numpy arrays (and may do so in the future), but for an initial and clear implementation, having a dictionary helps us keep track of all the derivatives at once.
3. Overriding each of the methods will allow fast and easy computation of the derivatives and their values.  Storing the variable names in the dictionary will also allow us to simultaneously keep track of all partials.

Last, we have an argument _direction_ to the get\_gradient method, which can be used to extract a directional derivative.  While this is not fully implemented in our pseudocode (as it is merely pseudocode), the user could extract a directional derivative by specifying f.get_der(direction={'x': 1, 'y': 3}).

In the future, we wish to have an option for users to input the value later, during the calculation of the derivative (as opposed to first specifying the value), and we talked to Daniel at OH about this idea, but for now, we keep the implementation relatively simple to ensure our code is up and running.

Pseudocode for this class is included below:

```python
import numpy as np

class DualNumber():
    '''
    Description: a class to hold dual number representations of vectors/scalars.
    '''
    
    def __init__(self, real_part, imag_part=1, variables):
        if type(real_part) or type(imag_part) == str:
            # Ensure that the input into constructor is valid
            raise(ValueError('The input cannot be string'))
            
        elif len(real_part) != len(image_part):
            raise ValueError
            
        else:
            # intialization should just have one variable in init
            self.val=dict(zip(variables, real_part))
            self.der=dict(zip(variables, np.ones(len(real_part)) # set initial imaginary part to 1, since this represents the derivative of x
            self.variables = list(variables)
            
    
    def __add__(self, second_var):
        if self.variables == second_var.variables:
            #Override default adding with dunder method
            self.val = self.val + second_var.val
            self.der = self.der + second_var.der
        else:
            # the value should still be the sum of both functions, even if both are different variables
            self.val = self.val + second_var.val
                              
            for each variable in self and second_var:
                self.der[variable] = self.der[variable] + second_var.der[variable]
            for variables only in second_var:
                self.der[variable] = second_var.val[variable]
            
        
    def __radd__(self,second_var):
        # This function should be able to handle left and right add
        
    def __mul__(self,second_var):
        #Similarly, we can override mutiply
    def __rmul__(self,second_var):
        #Similarly, we can override right mutiply
        
    def __sub__(self,second_var):
        #Similarly, we can override subtraction
    def __rsub__(self,second_var):
        #Similarly, we can override right subtraction
        
    def __div__(self, second_var):
    def __rdiv__(self, second_var):
                              
    def get_value(self):
        # sum the values of the function at each variable
        return sum(self.val.values())
                              
    def get_der(self, variable, direction=None):
        if direction is None:
            return self.der[variable]
        else:
            return directional derivative
            
    
```

### ElemFunctions()

We will also implement a class ElemFunctions() which will inherit from DualNumber.  Here we will allow the user to 
Note that we should be careful about how the function is called when we have multiple variables.  For example, suppose the user defines:

1. x = DualNumber(5, 'x')
2. y = DualNumber(6, 'y')
3. f = ElemFunctions.exp(x*y)

Then, the DualNumber object $xy$ will have a dictionary of derivatives for $x,y$, and we must augment each of these derivatives in the exp method.

Recall, if $f(x, y) = e^{xy}$, then $f_y(x,y) = xe^{xy}$, so we should set f.der['y'] = np.exp(xy.val)* xy.der['y'].  Again, this design allows simplicity and clarity in design, as the user only has to initialize each variable once as a DualNumber object and then may take an arbitary combination of elementary functions and operators on this set of variables to get more complicated functions.  This allows us to find gradients of scalar-valued vector functions.

We implement this process below:


```python
import numpy as np

class ElemFunctions(DualNumber):

    def exp(self, x):
        # first: assert x is DualNumber (which could also be AutoDiff object)
        self.value= np.exp(x.value)
        for each variable:
            self.der[variable] = np.exp(x.value)*x.der[variable]
            
    def sin(self,x):
        #Similarly, we can give user defined sin function, 
        self.value= np.sin(x.value)
        for each variable:
            self.der[variable] = np.cos(x.value)*x.der[variable]
        
    def cos(self,x):#Similarly, we can give user defined cos function, 
    def log(self,x):#Similarly, we can give user defined log function, 
        
```

We will also have a class Parallelized(), which will allow us to compute the Jacobian of vector-valued functions of vectors.  

We will not outline this class in pseudocode but rather in words.  Suppose our input is $n$ dimensional, then Parallelized will construct $n$ DualNumber objects, one for each of the inputs, by looping over each of the inputs.  It will construct the gradient of each variable in the DualNumber (and ElemFunction()) class and construct the Jacobian matrix by concatenating each of these $n$ gradients (each one has length $m$ where $m$ is output dimension) into one matrix.

To get directional gradients, we will make sure that the user inputs an $n$-dimensional (where $n$ is the number of inputs).  We will also check the dimension of the Jacobian, input vector, output vector, and directional derivative vector $\vec{p}$ to make sure these all align.

Last, we will also have a class Visualizations(), in which we hope to implement a few computational graph visualizations of automatic differentiation.

## External Dependencies
We will try to limit our external dependencies, but we will have to use **numpy** (and possibly **math**) for our elementary functions and array/matrix manipulation.  We will also use **pytest** and **pytest-cov** for test suites.